In [6]:
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession
import warnings

#Supress Warnings
warnings.filterwarnings('ignore')

#Data Creation

In [7]:
#Pandas onSpark Series
s = ps.Series([1,2,3,np.nan,5,6,np.nan,np.nan,9])
s

0    1.0
1    2.0
2    3.0
3    NaN
4    5.0
5    6.0
6    NaN
7    NaN
8    9.0
dtype: float64

In [12]:
#Pandas on Spark DataFrame
psdf = ps.DataFrame(
    {
    'color':['red','black','grey'],
    'num':[2,3,1],
    'amount':[500,250,450]
},
index = [10,20,30])
psdf

,color,num,amount
10,red,2,500
20,black,3,250
30,grey,1,450


In [14]:
# Pands DF with Numpy
dates = pd.date_range('20230427', periods=6)
dates

DatetimeIndex(['2023-04-27', '2023-04-28', '2023-04-29', '2023-04-30',
               '2023-05-01', '2023-05-02'],
              dtype='datetime64[ns]', freq='D')